In [1]:
%matplotlib inline

### Load Meert+15 catalog

In [2]:
from rescale_dc2.load_catalogs import read_meert_catalog
meert15 = read_meert_catalog()
print(meert15.keys())

['objid', 'z', 'ra', 'dec', 'kcorr', 'bulge_to_total_rband', 'r50_magr_kpc', 'r50_magr_disk_kpc', 'r50_magr_bulge_kpc']


### Load 5-band flux catalog

In [3]:
import numpy as np 
from astropy.table import Table
import os
dirname = "/Users/aphearin/work/SDSS/dr7_five_fluxes"
basename1 = "MyTable_1a_aphearin.csv"
fname1 = os.path.join(dirname, basename1)

dr7 = Table.read(fname1, format='csv')
dr7.sort('objID')

unique_vals, indices = np.unique(dr7['objID'], return_index=True)

dr7 = dr7[indices]

dr7 = dr7[dr7['z']>0.01]
print(dr7.keys())

['specObjID', 'objID', 'ra', 'dec', 'z', 'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z', 'extinction_u', 'extinction_g', 'extinction_r', 'extinction_i', 'extinction_z']


### Add Meert+15 keys to 5-band flux catalog by crossmatching on `objID`

In [4]:
from halotools.utils import crossmatch 

idxA, idxB = crossmatch(dr7['objID'], meert15['objid'])

meert_keys_to_inherit = ('kcorr', 'bulge_to_total_rband', 'r50_magr_kpc', 
                  'r50_magr_disk_kpc', 'r50_magr_bulge_kpc')

for key in meert_keys_to_inherit:
    dr7[key] = np.nan
    dr7[key][idxA] = meert15[key][idxB]

### Load DR10 volume-limited catalog with $M_{\ast}$ and ${\rm sSFR}$

In [5]:
dirname = "/Users/aphearin/work/sdss/cross_matched_catalogs"
basename = "umachine_sdss_dr10_value_added_bt_corrected_dr7_objID.hdf5"
dr10_fname = os.path.join(dirname, basename)

dr10 = Table.read(dr10_fname, path='data')

### Add Meert+15 keys and 5-band flux keys from dr7 to dr10

In [6]:
bands = ('u', 'g', 'r', 'i', 'z')
keys = ['extinction_'+band for band in bands]
modelMag_keys = ['modelMag_'+band for band in bands]
keys.extend(modelMag_keys)
keys.extend(meert_keys_to_inherit)

idxA, idxB = crossmatch(dr10['dr7_objID'], dr7['objID'])

for key in keys:
    dr10[key] = np.nan
    dr10[key][idxA] = dr7[key][idxB]

In [7]:
keys_to_delete = ('bptclass', 'specsfr', 'sm_fib', 'sfr_fib', 'specsfr_fib', 'withinr', 
                 'eligiblehost', 'eligiblehost2', 'd4000_sub', 'd4000_n_sub', 'logM_mendel13', 
                 'pps_mendel13', 'type_mendel13', 'deltaBD_mendel13', 'logMB_mendel13', 
                 'logMD_mendel13', 'd4000_n')
for key in keys_to_delete:
    del dr10[key]

In [8]:
print(dr10.keys())

['ra', 'dec', 'z', 'sm', 'sfr', 'sm_complete', 'd4000', 'd4000_n', 'has_match', 'dr7_objID', 'extinction_u', 'extinction_g', 'extinction_r', 'extinction_i', 'extinction_z', 'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z', 'kcorr', 'bulge_to_total_rband', 'r50_magr_kpc', 'r50_magr_disk_kpc', 'r50_magr_bulge_kpc']
